**Rule Versus Complexity Diagnostic** </br></br>
The Rule vs. Complexity Diagnostic module is designed to help users determine whether rule-based logic can effectively address an ethical challenge or if a more complex solution is required. Using scenarios that test the boundaries between simple principles and nuanced reasoning, users practice the meta-skill of recognizing when straightforward ethical rules apply versus when deeper analysis is needed. Each scenario presents a fairness dilemma, asking users to choose between applying a simple rule or engaging in complex reasoning, with immediate feedback explaining the reasoning behind each choice. This develops the crucial ability to self-assess the complexity of ethical situations before attempting to solve them.

In [ ]:
#@title pip install
from IPython.display import clear_output

!pip install ipywidgets
clear_output()  # Hides the output after the cell runs

In [ ]:
#@title Imports
import random
import time
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, VBox, HBox, HTML, Layout, RadioButtons

In [ ]:
#@title Scenarios
scenarios = [
    # SIMPLE RULE scenarios - clear principles apply
    {
        "text": "Your hiring algorithm shows 95% accuracy for all demographic groups. Legal team asks: 'Is this fair?'",
        "is_complex": False,
        "simple_rule": "Equal performance across groups = fair",
        "explanation": "When accuracy is equal across groups, this is straightforward fairness. Simple rule applies.",
        "complexity_trap": "Some might overthink this, but equal performance IS the goal."
    },
    {
        "text": "Algorithm uses protected characteristics (race, gender) directly as input features for loan decisions.",
        "is_complex": False,
        "simple_rule": "Never use protected characteristics directly",
        "explanation": "Direct use of protected characteristics is clearly discriminatory. Simple rule applies.",
        "complexity_trap": "No need for complex analysis - this is straightforward illegal discrimination."
    },
    {
        "text": "AI system randomly assigns loan approvals regardless of any applicant information.",
        "is_complex": False,
        "simple_rule": "Random decisions are fair but useless",
        "explanation": "Pure randomness is technically fair but completely defeats the purpose. Simple rule applies.",
        "complexity_trap": "While random is 'fair,' it's obviously not a viable business solution."
    },
    {
        "text": "Facial recognition works perfectly for everyone regardless of age, race, or gender.",
        "is_complex": False,
        "simple_rule": "Equal performance = no bias problem",
        "explanation": "If performance is truly equal across all groups, there's no bias issue. Simple rule applies.",
        "complexity_trap": "Don't overcomplicate success - equal performance means the system is working fairly."
    },
    # COMPLEX REASONING scenarios - competing values, trade-offs, stakeholder conflicts
    {
        "text": "Hiring algorithm: 85% accuracy for men, 82% accuracy for women. Legal says 'fix it,' Engineering says 'close enough,' HR says 'focus on candidate experience.'",
        "is_complex": True,
        "simple_rule": None,
        "explanation": "Competing stakeholder priorities, unclear fairness standards, and trade-offs between accuracy and equity require complex reasoning.",
        "complexity_factors": ["Multiple stakeholder views", "Unclear fairness threshold", "Accuracy vs. equity trade-off"]
    },
    {
        "text": "Medical AI: 90% accuracy for common diseases, 70% for rare diseases. Rare disease patients are predominantly from minority communities.",
        "is_complex": True,
        "simple_rule": None,
        "explanation": "Medical necessity vs. equity, technical limitations vs. fairness obligations, and intersectional impacts require complex reasoning.",
        "complexity_factors": ["Medical vs. social priorities", "Technical constraints", "Intersectional impacts"]
    },
    {
        "text": "Loan algorithm has equal accuracy across racial groups but approves fewer loans in low-income neighborhoods (which are predominantly minority).",
        "is_complex": True,
        "simple_rule": None,
        "explanation": "Equal treatment vs. equal outcomes, individual vs. community impacts, and historical context require complex reasoning.",
        "complexity_factors": ["Individual vs. group fairness", "Historical context", "Economic vs. social factors"]
    },
    {
        "text": "Your 'fair' algorithm reduces bias but also reduces overall hiring by 30%, potentially hurting company competitiveness.",
        "is_complex": True,
        "simple_rule": None,
        "explanation": "Fairness vs. business viability, short-term vs. long-term impacts, and stakeholder trade-offs require complex reasoning.",
        "complexity_factors": ["Business vs. ethical priorities", "Short vs. long-term thinking", "Multiple stakeholder impacts"]
    }
]

In [ ]:
#@title Rule Complexity
class RuleComplexityDiagnostic:
    def __init__(self, scenarios_list):
        self.scenarios = scenarios_list.copy()
        self.current_scenario = 0
        self.score = 0
        self.responses = []
        self.start_time = None
        self.response_times = []

        random.shuffle(self.scenarios)

        # Create widgets
        self.scenario_display = HTML(
            value="<h3>Rule vs. Complexity Diagnostic</h3>",
            layout=Layout(text_align='center')
        )

        self.rule_button = Button(
            description='Simple Rule Applies',
            button_style='success',
            layout=Layout(width='250px', height='60px', margin='5px')
        )

        self.complex_button = Button(
            description='Needs Complex Reasoning',
            button_style='warning',
            layout=Layout(width='250px', height='60px', margin='5px')
        )

        self.next_button = Button(
            description='NEXT →',
            button_style='',
            layout=Layout(width='100px', height='30px', margin='10px')
        )

        self.feedback_display = HTML(layout=Layout(text_align='center'))
        self.progress_display = HTML(layout=Layout(text_align='center'))

        # Button clicks
        self.rule_button.on_click(lambda b: self.handle_response(False))
        self.complex_button.on_click(lambda b: self.handle_response(True))
        self.next_button.on_click(lambda b: self.next_scenario())

        # Layout
        answer_buttons = HBox([self.rule_button, self.complex_button], layout=Layout(justify_content='center'))
        next_button_box = HBox([self.next_button], layout=Layout(justify_content='center'))

        self.widget = VBox([
            self.scenario_display,
            answer_buttons,
            self.feedback_display,
            next_button_box,
            self.progress_display
        ])

    def start_diagnostic(self, num_scenarios=8):
        self.scenarios = self.scenarios[:num_scenarios]
        self.current_scenario = 0
        self.score = 0
        self.responses = []
        self.response_times = []
        self.show_scenario()
        return self.widget

    def show_scenario(self):
        if self.current_scenario >= len(self.scenarios):
            self.show_results()
            return

        scenario = self.scenarios[self.current_scenario]
        self.start_time = time.time()

        self.scenario_display.value = f"""
        <div style='border: 2px solid #ddd; padding: 20px; border-radius: 10px; background: #f9f9f9; text-align: center;'>
            <h3>Scenario {self.current_scenario + 1} of {len(self.scenarios)}</h3>
            <p style='font-size: 16px; line-height: 1.5;'><strong>{scenario['text']}</strong></p>
            <p style='color: #666;'><em>Can you apply a simple rule here, or does this need complex reasoning?</em></p>
        </div>
        """
        self.feedback_display.value = ""
        self.update_progress()

    def handle_response(self, user_says_complex):
        if self.start_time is None:
            return

        response_time = time.time() - self.start_time
        scenario = self.scenarios[self.current_scenario]
        correct = user_says_complex == scenario['is_complex']

        if correct:
            self.score += 1

        self.responses.append({
            'correct': correct,
            'response_time': response_time
        })
        self.response_times.append(response_time)

        # Show feedback
        color = "green" if correct else "red"
        text = "✓ Correct!" if correct else "✗ Incorrect"

        if scenario['is_complex']:
            explanation = scenario['explanation']
        else:
            explanation = f"Simple rule: {scenario['simple_rule']}. {scenario['explanation']}"

        self.feedback_display.value = f"""
        <div style='border: 2px solid {color}; padding: 15px; border-radius: 8px; margin: 10px 0; text-align: center;'>
            <h4 style='color: {color};'>{text}</h4>
            <p>{explanation}</p>
        </div>
        """

        self.start_time = None
        self.update_progress()

    def next_scenario(self):
        self.current_scenario += 1
        self.show_scenario()

    def update_progress(self):
        if len(self.responses) > 0:
            accuracy = self.score / len(self.responses) * 100
            avg_time = sum(self.response_times) / len(self.response_times)
        else:
            accuracy = avg_time = 0

        self.progress_display.value = f"""
        <div style='background: #e8f4fd; padding: 10px; border-radius: 5px; text-align: center;'>
            <strong>Progress:</strong> {len(self.responses)}/{len(self.scenarios)} |
            <strong>Accuracy:</strong> {accuracy:.1f}%
        </div>
        """

    def show_results(self):
        accuracy = self.score / len(self.scenarios) * 100

        self.scenario_display.value = f"""
        <div style='border: 3px solid #4CAF50; padding: 25px; border-radius: 15px; background: #f8fff8; text-align: center;'>
            <h2 style='color: #4CAF50;'>Diagnostic Complete! 🎯</h2>
            <p><strong>Accuracy:</strong> {accuracy:.1f}% ({self.score}/{len(self.scenarios)})</p>
        </div>
        """

        # Hide buttons
        self.rule_button.layout.display = 'none'
        self.complex_button.layout.display = 'none'
        self.next_button.layout.display = 'none'
        self.feedback_display.value = ""

In [ ]:
#@title Start Diagnostic Tool
# Start the diagnostic
diagnostic = RuleComplexityDiagnostic(scenarios)
widget = diagnostic.start_diagnostic(8)  # Use 8 scenarios
display(widget)